In [20]:
import os
os.environ["OPENAI_API_KEY"] = "Place your key here"

In [26]:
# 📦 Imports
import os
import json
import re
import time
import nest_asyncio
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.chat_models import ChatOpenAI
from playwright.sync_api import sync_playwright

# ✅ Required for Playwright in Jupyter
nest_asyncio.apply()

# 🔑 Set your OpenAI API Key
os.environ["OPENAI_API_KEY"] = "Place your key here"

# 🧠 LLM setup
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# 🧠 Prompt template for MCP-style JSON actions
mcp_prompt = PromptTemplate.from_template("""
You're a browser automation assistant. Convert the following task into a **pure JSON array** of Playwright MCP `actions`.

TASK: {task}

RESPONSE FORMAT (must match):
[
  {{ "type": "goto", "url": "..." }},
  {{ "type": "wait", "time": ... }},
  ...
]
Return only the JSON. No markdown. No explanation.
""")

mcp_chain = LLMChain(llm=llm, prompt=mcp_prompt)

In [28]:
# 💬 Example user task
task_prompt = (
    "Go to Partha Sai Guttikonda's portfolio page at https://guttikondaparthasai.info/, "
    "wait for 5 seconds, and take a full-page screenshot named 'partha_screenshot.png'."
)

generated = mcp_chain.run(task=task_prompt)

# 🧼 Parse JSON from LLM output
match = re.search(r"\[\s*{.*?}\s*\]", generated, re.DOTALL)
if match:
    cleaned_json = match.group(0)
    try:
        actions = json.loads(cleaned_json)
    except json.JSONDecodeError as e:
        print("JSON Decode Error:", e)
        print("Original Output:\n", generated)
        raise
else:
    print("❌ Failed to parse JSON. Full LLM output:\n", generated)
    raise ValueError("No valid JSON array found.")

In [30]:
# 🤖 Playwright Executor for Jupyter
def execute_actions_notebook(actions):
    p = sync_playwright().start()
    browser = p.chromium.launch(headless=True)
    page = browser.new_page()

    for action in actions:
        action_type = action.get("type")
        try:
            if action_type == "goto":
                print(f"🔗 Navigating to {action['url']}")
                page.goto(action["url"])

            elif action_type == "wait":
                print(f"⏱ Waiting for {action['time']} seconds")
                time.sleep(action["time"])

            elif action_type == "click":
                print(f"🖱 Clicking element {action['selector']}")
                page.click(action["selector"])

            elif action_type == "fill":
                print(f"📝 Filling {action['selector']} with '{action['value']}'")
                page.fill(action["selector"], action["value"])

            elif action_type == "type":
                print(f"⌨️ Typing in {action['selector']} with delay {action.get('delay', 50)}ms")
                page.type(action["selector"], action["value"], delay=action.get("delay", 50))

            elif action_type == "select":
                print(f"🔽 Selecting {action['value']} in {action['selector']}")
                page.select_option(action["selector"], action["value"])

            elif action_type == "screenshot":
                print(f"📸 Taking screenshot and saving to {action['path']}")
                page.screenshot(path=action["path"], full_page=action.get("fullPage", False))

            else:
                print(f"⚠️ Unsupported action type: {action_type}")

        except Exception as e:
            print(f"❌ Error while performing {action_type}: {e}")

    browser.close()
    p.stop()

# 🏃‍♂️ Run the Executor
execute_actions_notebook(actions)

Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.